In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Demo-Cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print("Creating a new compute cluster...")
    compute_config =  AmlCompute.provisioning_configuration(vm_size='STANDARD_D2S_V3', max_nodes=4, min_nodes=0, vm_priority='lowpriority') 
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
    
#aml_compute.wait_for_completion(show_output=True)
aml_compute.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)
#compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)


# use get_status() to get a detailed status for the current cluster. 
print(aml_compute.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform , choice
from azureml.train.estimator import Estimator
import os

# Specify parameter sampler
paraSampler = RandomParameterSampling(
             {
            "--C": uniform(0.1, 1.0),
              } )


# Specify a Policy
policyBandit = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=aml_compute, entry_script="train.py",
              script_params={'--C':1.0, '--max_iter':100})    

# Create a SKLearn estimator for use with train.py
#est =  Estimator(source_directory=os.path.join('.'),
#                      entry_script='train.py',
#                      compute_target=aml_compute)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                    estimator=est,
                    hyperparameter_sampling=paraSampler,
                    policy=policyBandit,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_concurrent_runs = 4,    
                    max_total_runs=2)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

#exp = Experiment(workspace=ws, name='tuning')
#run = exp.submit(hyperdrive_config)

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

#run.wait_for_completion(show_output=True)

### YOUR CODE HERE ###